In [1]:
import pandas as pd
import gdown
import re
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import gc
import time
import pytz
import datetime
import logging
from tqdm import tqdm
import traceback

In [2]:
def driver_setup():# Assigning a Headless Firefox Driver
  options = webdriver.FirefoxOptions()
  options.binary_location = 'C:/Program Files/Mozilla Firefox/firefox.exe'  # Path to Chrome executable
  options.add_argument('E:/Programs/geckodriver.exe')
  # options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Firefox(options=options)
  return driver

def save_data_to_disk(data, filename):
    if os.path.isfile(filename):
        existing_df = pd.read_csv(filename)
        data = pd.concat([existing_df, data], ignore_index=True)
        os.remove(filename)
        data.to_csv(filename,index=False)
    else:
        data.to_csv(filename, mode='w', header=True, index=False)

In [3]:
i = 0
driver = driver_setup()
url = 'https://www.indeed.com/jobs?q=USA&sc=0kf%3Ajt%28fulltime%29%3B&vjk=d8934d58ce092ffe'
driver.get(url)

The version of firefox cannot be detected. Trying with latest driver version


In [4]:
save = 1
job_title = None
company = None
salary = None
location = None 
description = None
posted_on = None
current_date = None 

In [8]:
while True:
#### page loop
    #### elemental loop    
    time.sleep(4)
    
    li_elements = driver.find_elements(By.CSS_SELECTOR, '.css-zu9cdh > li')
    for i in range(len(li_elements)): # elemental loop
        time.sleep(2)
        data = {}
        try:
            li_elements[i].find_element(By.CSS_SELECTOR, 'a').click()
        except:
            continue
            
        try:
            salary = li_elements[i].find_element(By.CSS_SELECTOR, '.css-1ihavw2').text
            if salary == 'Full-time':
                continue
        except:
            continue
        
        try:
            job_title = ' '.join(li_elements[i].find_element(By.CSS_SELECTOR, 'h2').text.split('\n'))
        except:
            None
        try:
            company = li_elements[i].find_element(By.CLASS_NAME, 'companyName').text
        except:
            None
        try:
            location = li_elements[i].find_element(By.CLASS_NAME, "companyLocation").text
        except:
            None
        try:
            posted_on = li_elements[i].find_element(By.CLASS_NAME, "date").text.split('\n')[1]
        except:
            None
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.jobsearch-embeddedBody')))
        except:
            None
        skills = []
        try:
            mosaic_element = driver.find_element(By.ID, 'js-match-insights-provider')
            profile_insights_header = mosaic_element.find_element(By.XPATH, '//h2[normalize-space(text())="Profile insights"]')
            if profile_insights_header:
                skills_header = mosaic_element.find_element(By.XPATH, '//h2[normalize-space(text())="Skills"]')
                if skills_header:
                    skills_list = mosaic_element.find_elements(By.CSS_SELECTOR, '.js-match-insights-provider-cmd4as li')
                    skills = [skill.text for skill in skills_list]
        except Exception as e:
            None
            
        try:
            description = driver.find_element(By.ID, 'jobDescriptionText').text
        except:
            None 
            
        current_date = datetime.datetime.now().date()
        data = {
            'job_title':job_title,
            'company_name':company,
            'salary':salary,
            'location':location,
            'description':description,
            'posted_on':posted_on,
            'current_date':current_date
        }
        for i in range(len(skills)):
            data['skill'+str(i+1)]=skills[i]
        
        if data != {}:
            print(f'\rProgress: {save}', end='')
            save+=1
            tempdf = pd.DataFrame(data, index=[0])
            tempfilename = 'indeed_raw.csv'

            save_data_to_disk(tempdf, tempfilename)
            # make all nans.
            tempdf = pd.DataFrame()
        job_title = None
        company = None
        salary = None
        location = None 
        description = None
        posted_on = None
        current_date = None    
        skills = []

    try:
        next_button = driver.find_element(By.XPATH, '//a[@data-testid="pagination-page-next"]')
        next_button.click()
    except Exception as e:
        break 

In [7]:
df = pd.read_csv(r'indeed_raw.csv')
df['experience'] = 'entry level'
# df['experience'] = 'mid level'
# df['experience'] = 'senior level'
os.remove(r'indeed_raw.csv')
# tempfilename = 'indeed_energy_roles.csv'
tempfilename = 'indeed_store.csv'
save_data_to_disk(df, tempfilename)

FileNotFoundError: [Errno 2] No such file or directory: 'indeed_raw.csv'

In [21]:
df = pd.read_csv(tempfilename)

In [25]:
df['current_date'].unique()

array(['2023-10-18', '2023-10-19', '2023-10-20', '2023-10-23',
       '2023-10-25', '2023-10-26', '2023-10-27', '2023-10-31',
       '2023-11-01', '2023-11-02', '2023-11-03', '2023-11-06',
       '2023-11-07', '2023-11-08', '2023-11-10', '2023-11-13',
       '2023-11-14', '2023-11-15'], dtype=object)

In [15]:

subsetcols = df.columns.to_list()
subsetcols.remove('current_date')
subsetcols.remove('posted_on')
df.drop_duplicates(subset=subsetcols, inplace=True)
df.reset_index(drop=True,inplace=True)
# df.to_csv('Naukri_raw.csv',index=False)

In [16]:
df.shape

(11951, 17)

In [11]:
current_time = datetime.datetime.now()
filename = 'Indeed_Extract_'+ str(current_time.date()).replace('-', '')+'.xlsx'
# filename = 'Indeed_Energy_Roles'+ str(current_time.date()).replace('-', '')+'.xlsx'
df.to_excel(filename,index = False)

In [12]:
'https://onedrive.live.com/?id=F87FC3AA6EB46E0%21205009&cid=0F87FC3AA6EB46E0'

'https://onedrive.live.com/?id=F87FC3AA6EB46E0%21205009&cid=0F87FC3AA6EB46E0'